<a href="https://colab.research.google.com/github/Kalazclint/HdayBuzz-ContentClassifier/blob/main/RseNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
csv_file = '/content/drive/My Drive/dataset/train.csv'


In [19]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim


In [16]:
df = pd.read_csv(csv_file)
df.head()


,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [17]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.class_to_idx = {class_name: i for i, class_name in enumerate(self.df['Class'].unique())}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')  # Convert image to RGB
        label = self.class_to_idx[self.df.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label


In [20]:

# Define the transformations: resizing the images to 128x128 and converting them to tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Create datasets
dataset = CustomImageDataset(csv_file= '/content/drive/My Drive/dataset/train.csv', img_dir='/content/drive/My Drive/dataset/train/', transform=transform)

# Split data into training, validation, and test sets and create dataloaders
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [21]:
import torch.nn as nn

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = conv3x3(out_channels, out_channels, stride=stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet50(nn.Module):
    def __init__(self, block, layers, num_classes=6):
        super(ResNet50, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

model = ResNet50(BottleneckBlock, [3, 4, 6, 3])


In [24]:
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
num_epochs = 20  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/20], Step [10/142], Loss: 1.7043
Epoch [1/20], Step [20/142], Loss: 1.6746
Epoch [1/20], Step [30/142], Loss: 1.7615
Epoch [1/20], Step [40/142], Loss: 1.2603
Epoch [1/20], Step [50/142], Loss: 1.3220
Epoch [1/20], Step [60/142], Loss: 1.1371
Epoch [1/20], Step [70/142], Loss: 1.2211
Epoch [1/20], Step [80/142], Loss: 1.2218
Epoch [1/20], Step [90/142], Loss: 1.1833
Epoch [1/20], Step [100/142], Loss: 1.2981
Epoch [1/20], Step [110/142], Loss: 1.1089
Epoch [1/20], Step [120/142], Loss: 1.4154
Epoch [1/20], Step [130/142], Loss: 1.3973
Epoch [1/20], Step [140/142], Loss: 0.8540
Epoch [2/20], Step [10/142], Loss: 1.4337
Epoch [2/20], Step [20/142], Loss: 1.2060
Epoch [2/20], Step [30/142], Loss: 1.1774
Epoch [2/20], Step [40/142], Loss: 1.2728
Epoch [2/20], Step [50/142], Loss: 1.3231
Epoch [2/20], Step [60/142], Loss: 1.3179
Epoch [2/20], Step [70/142], Loss: 1.3836
Epoch [2/20], Step [80/142], Loss: 1.0898
Epoch [2/20], Step [90/142], Loss: 1.0904
Epoch [2/20], Step [100/142],

In [26]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 67.01%


In [27]:
import torch.optim as optim

# Define the criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training Loop
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f"[Epoch {epoch+1}] Loss: {running_loss/i:.3f}, Training Accuracy: {train_accuracy:.2f}%")

# Testing Loop
correct_test = 0
total_test = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")


[Epoch 1] Loss: 0.368, Training Accuracy: 87.37%
[Epoch 2] Loss: 0.224, Training Accuracy: 92.47%
[Epoch 3] Loss: 0.204, Training Accuracy: 93.40%
[Epoch 4] Loss: 0.187, Training Accuracy: 93.62%
[Epoch 5] Loss: 0.179, Training Accuracy: 94.13%
[Epoch 6] Loss: 0.179, Training Accuracy: 93.99%
[Epoch 7] Loss: 0.159, Training Accuracy: 95.43%
[Epoch 8] Loss: 0.152, Training Accuracy: 95.01%
[Epoch 9] Loss: 0.152, Training Accuracy: 94.99%
[Epoch 10] Loss: 0.138, Training Accuracy: 95.56%
[Epoch 11] Loss: 0.115, Training Accuracy: 96.62%
[Epoch 12] Loss: 0.112, Training Accuracy: 96.55%
[Epoch 13] Loss: 0.107, Training Accuracy: 96.80%
[Epoch 14] Loss: 0.110, Training Accuracy: 96.75%
[Epoch 15] Loss: 0.103, Training Accuracy: 96.86%
[Epoch 16] Loss: 0.092, Training Accuracy: 97.31%
[Epoch 17] Loss: 0.091, Training Accuracy: 97.15%
[Epoch 18] Loss: 0.096, Training Accuracy: 97.20%
[Epoch 19] Loss: 0.078, Training Accuracy: 97.81%
[Epoch 20] Loss: 0.093, Training Accuracy: 97.02%
Test Accu